In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, datasets, models
from sklearn.model_selection import KFold
import numpy as np
import os

In [6]:
import warnings

warnings.filterwarnings(action='ignore')

In [3]:
data_dir = 'fake_licenseplate'
batch_size = 32
num_epochs = 10
k_folds = 5
learning_rate = 0.001

transform = transforms.Compose([
    transforms.Resize((144, 48)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(SimpleCNN, self).__init__()
        self.features = models.resnet18(pretrained=True)
        self.features.fc = nn.Linear(self.features.fc.in_features, num_classes)

    def forward(self, x):
        x = self.features(x)
        return x

In [7]:
kfold = KFold(n_splits=k_folds, shuffle=True)

results = {}

for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
    print(f'Fold {fold + 1}')

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)

    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_subsampler)
    val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_subsampler)

    model = SimpleCNN(num_classes=2)
    model = model.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        for batch in train_loader:
            images, labels = batch
            images, labels = images.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')), labels.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_loader:
                images, labels = batch
                images, labels = images.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu')), labels.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = correct / total
        print(f'Fold {fold + 1}, Epoch {epoch + 1}, Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}')

    results[fold] = val_acc

print(f'K-Fold Cross Validation results: {results}')
print(f'Average Accuracy: {np.mean(list(results.values())):.4f}')

Fold 1
Fold 1, Epoch 1, Loss: 6.3347, Accuracy: 0.5455
Fold 1, Epoch 2, Loss: 25.1126, Accuracy: 0.2909
Fold 1, Epoch 3, Loss: 3.0921, Accuracy: 0.6364
Fold 1, Epoch 4, Loss: 1.4152, Accuracy: 0.7455
Fold 1, Epoch 5, Loss: 0.6714, Accuracy: 0.9091
Fold 1, Epoch 6, Loss: 0.0578, Accuracy: 0.9818
Fold 1, Epoch 7, Loss: 0.0073, Accuracy: 1.0000
Fold 1, Epoch 8, Loss: 0.0030, Accuracy: 1.0000
Fold 1, Epoch 9, Loss: 0.0018, Accuracy: 1.0000
Fold 1, Epoch 10, Loss: 0.0020, Accuracy: 1.0000
Fold 2
Fold 2, Epoch 1, Loss: 9.9773, Accuracy: 0.4727
Fold 2, Epoch 2, Loss: 0.4353, Accuracy: 0.9455
Fold 2, Epoch 3, Loss: 1.0721, Accuracy: 0.8545
Fold 2, Epoch 4, Loss: 0.3655, Accuracy: 0.9273
Fold 2, Epoch 5, Loss: 0.0182, Accuracy: 0.9818
Fold 2, Epoch 6, Loss: 0.8706, Accuracy: 0.8000
Fold 2, Epoch 7, Loss: 0.0044, Accuracy: 1.0000
Fold 2, Epoch 8, Loss: 0.0017, Accuracy: 1.0000
Fold 2, Epoch 9, Loss: 0.0021, Accuracy: 1.0000
Fold 2, Epoch 10, Loss: 0.0016, Accuracy: 1.0000
Fold 3
Fold 3, Epoch 1,